## Loading Libraries

In [2]:
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd  # import pandas library and give it an alias of pd
from scipy.stats import norm, kurtosis
import seaborn as sns
import math
import plotly.express as px
from matplotlib.ticker import PercentFormatter
from datetime import timedelta, date
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot
from xgboost import XGBClassifier
from pandas import read_csv
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from pandas import read_csv
from sklearn.feature_selection import RFE
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import scikitplot as skplt
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot

C:\Users\92306\anaconda3\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Loading Data

In [3]:
sales1 = pd.read_excel (r'/Users/92306/OneDrive/Documents/sales1.xlsx') # old sales file
sales2 = pd.read_excel (r'/Users/92306/OneDrive/Documents/sales2.xlsx') # any new sales data should be added or renamed as sales2.xlsx
account = pd.read_excel (r'/Users/92306/OneDrive/Documents/account.xlsx')
top_sellers = pd.read_excel (r'/Users/92306/OneDrive/Documents/top_selling_items.xlsx')
contact = pd.read_excel (r'/Users/92306/OneDrive/Documents/contact.xlsx')
#contact1 = pd.read_excel (r'/Users/92306/OneDrive/Documents/contact_features.xlsx')
family = pd.read_excel (r'/Users/92306/OneDrive/Documents/family.xlsx')


## Preparing Consolidated Sales Data

In [8]:
# Merging Sales Data
sales = pd.concat([sales1,sales2])
# Working towards new primary key Customer_time. This will be treated as a single observation
sales["customer_time"] = sales["Customer No"].astype(str) + sales["Posting Date"].astype(str)
# Filtering customers having customer no empty or blank
sales = sales[sales['Customer No'].apply(len) == 7]

## Consolidating the data transaction wise (Based on customer_time)
sales_consolidated = sales.groupby(['customer_time'])['Quantity Ordered','Line Item Amount'].sum().reset_index()
sales_consolidated1 = sales.groupby(['customer_time'])['Quantity Ordered','Line Item Amount'].sum().reset_index()
sales_consolidated1['Customer No'] = sales_consolidated1['customer_time'].str[:7]
sales_consolidated1['Posting Date'] = sales_consolidated1['customer_time'].str[7:]
sales_consolidated['Customer No'] = sales_consolidated['customer_time'].str[:7]
sales_consolidated['Posting Date'] = sales_consolidated['customer_time'].str[7:]

# Bringing next order/transaction date against the existing transaction

sales_consolidated.loc[sales_consolidated['Customer No'] == sales_consolidated['Customer No'].shift(-1),'next_order_date'] = sales_consolidated['Posting Date'].shift(-1) 
sales_consolidated.loc[sales_consolidated['next_order_date'].isnull(),'next_order_date'] = pd.to_datetime('today')
sales_consolidated['next_order_date'] = pd.to_datetime(sales_consolidated['next_order_date'])
sales_consolidated['Posting Date']=pd.to_datetime(sales_consolidated['Posting Date'])

#Computing time to order in days

sales_consolidated['time_to_order'] = sales_consolidated['next_order_date'] - sales_consolidated['Posting Date']
sales_consolidated['time_to_order'] = sales_consolidated['time_to_order']/np.timedelta64(1,'D')

#Forcing the minimum order time to be 30 days

sales_consolidated.loc[sales_consolidated['time_to_order'] < 30,'time_to_order'] = 30

# Assigning each customer an ID and counting the number of transactions for each customer 
sales_consolidated['block'] = (sales_consolidated['Customer No'] != sales_consolidated['Customer No'].shift(1)).astype(int).cumsum()
sales_consolidated['count'] = sales_consolidated.groupby((sales_consolidated['Customer No'] != sales_consolidated['Customer No'].shift(1)).cumsum()).cumcount()+1

<ipython-input-8-5afd33cf95ff>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sales_consolidated = sales.groupby(['customer_time'])['Quantity Ordered','Line Item Amount'].sum().reset_index()
<ipython-input-8-5afd33cf95ff>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sales_consolidated1 = sales.groupby(['customer_time'])['Quantity Ordered','Line Item Amount'].sum().reset_index()


### Churn Standard

In [9]:
sales_consolidated['cumulative_sum_time'] = sales_consolidated.groupby('Customer No')['time_to_order'].cumsum()
sales_consolidated.loc[sales_consolidated['Customer No'] == sales_consolidated['Customer No'].shift(1),'cumulative_sum_time_shifted'] = sales_consolidated['cumulative_sum_time'].shift(1) 
sales_consolidated.loc[sales_consolidated['Customer No'] == sales_consolidated['Customer No'].shift(1),'count_shifted'] = sales_consolidated['count'].shift(1) 
sales_consolidated.loc[sales_consolidated['count'] > 1, 'expected_purchase_window'] = sales_consolidated['cumulative_sum_time_shifted']/sales_consolidated['count_shifted']
sales_consolidated.drop(['cumulative_sum_time', 'cumulative_sum_time_shifted','count_shifted'], axis=1, inplace=True)

# Setting expected purchase window = 60 for the first transaction based on the graph above
sales_consolidated.loc[sales_consolidated['count'] == 1, 'expected_purchase_window'] = 60
# Computing time to churn standard based on static multiplier of 4
sales_consolidated['time_to_churn_standard'] = sales_consolidated['expected_purchase_window']*4

sales_consolidated.loc[sales_consolidated['time_to_order'] < sales_consolidated['time_to_churn_standard'], 'churn_standard'] = 0
sales_consolidated.loc[sales_consolidated['time_to_order'] >= sales_consolidated['time_to_churn_standard'], 'churn_standard'] = 1

Marking last transaction for all the customer and removing single transaction customers

In [10]:
m1 = ~sales_consolidated.duplicated(['Customer No'], keep='last')
m2 = sales_consolidated.duplicated(['Customer No'], keep=False)
m = m1 & m2
sales_consolidated.loc[m, 'last'] = 'true'

sales_consolidated = sales_consolidated[sales_consolidated.duplicated('block',keep=False)]

### Churn Modified & Churn Score

Making column time to churn modified and churn modified based on modified multiplier

In [11]:
sales_consolidated.loc[(sales_consolidated['expected_purchase_window'] > 0) & (sales_consolidated['expected_purchase_window'] <= 60), 'time_to_churn_modified'] = sales_consolidated['time_to_churn_standard']
sales_consolidated.loc[(sales_consolidated['expected_purchase_window'] > 60) & (sales_consolidated['expected_purchase_window'] <= 120), 'time_to_churn_modified'] = sales_consolidated['expected_purchase_window']*3
sales_consolidated.loc[(sales_consolidated['expected_purchase_window'] > 120) & (sales_consolidated['expected_purchase_window'] <= 240), 'time_to_churn_modified'] = sales_consolidated['expected_purchase_window']*2
sales_consolidated.loc[(sales_consolidated['expected_purchase_window'] > 240), 'time_to_churn_modified'] = sales_consolidated['expected_purchase_window']

sales_consolidated.loc[sales_consolidated['time_to_order'] < sales_consolidated['time_to_churn_modified'], 'churn_modified'] = 0
sales_consolidated.loc[sales_consolidated['time_to_order'] >= sales_consolidated['time_to_churn_modified'], 'churn_modified'] = 1


## Making churn score column. This basically tells us about the state of the customer at a particular point in time
sales_consolidated['churn_score'] = sales_consolidated['time_to_order']/sales_consolidated['time_to_churn_standard']*100
sales_consolidated.loc[sales_consolidated['churn_score'] > 100,'churn_score'] = 100

sales_consolidated.loc[(sales_consolidated['churn_score'] < 25),'churn alarm'] = 'Normal'
sales_consolidated.loc[(sales_consolidated['churn_score'] >= 25) & (sales_consolidated['churn_score'] < 50),'churn alarm'] = 'slightly concerning'
sales_consolidated.loc[(sales_consolidated['churn_score'] >= 50) & (sales_consolidated['churn_score'] < 75),'churn alarm'] = 'very concerning'
sales_consolidated.loc[(sales_consolidated['churn_score'] >=75),'churn alarm'] = 'critical'
sales_consolidated['year'] = pd.DatetimeIndex(sales_consolidated['Posting Date']).year


## Account Data Merging with Sales

In [13]:
account = account.rename(columns={"AccountNumber": "Customer No"})
## Bringing accounts data information in consolidated data
sales_account = pd.merge(sales_consolidated,
                 account,
                 on='Customer No', 
                 how='left')
## Making customer wise table side by side
sales_consolidated_lasttransaction = sales_account.loc[sales_account['last'] == 'true']

### Checking Churn Rate trend CLV wise and making CLV buckets
sales_account['cumulative_sum_revenue'] = sales_account.groupby('Customer No')['Line Item Amount'].cumsum()
sales_account.loc[(sales_account['cumulative_sum_revenue'] < 1000), 'revenue_bucket'] = 1
sales_account.loc[(sales_account['cumulative_sum_revenue'] >= 1000) & (sales_account['cumulative_sum_revenue'] < 2500), 'revenue_bucket'] = 2
sales_account.loc[(sales_account['cumulative_sum_revenue'] >= 2500) & (sales_account['cumulative_sum_revenue'] <= 5000), 'revenue_bucket'] = 3
sales_account.loc[(sales_account['cumulative_sum_revenue'] > 5000) & (sales_account['cumulative_sum_revenue'] <= 10000), 'revenue_bucket'] = 4                               
sales_account.loc[(sales_account['cumulative_sum_revenue'] > 10000), 'revenue_bucket'] = 5

###Checking Churn Rate as a function of number of transaction. 
labels_4 = ['first_quintile', 'second_quintile', 'third_quintile', 'fourth_quintile','fifth_quintile']
sales_account['count_group'] = pd.qcut(sales_account['count'],q=[0,0.2,0.4,0.6,0.8,1],labels=labels_4)

### Merging Company Revenue Ranges for better view
sales_account.loc[(sales_account['ZI_Company_Revenue_Range__c'] == 'Under $500,000') | (sales_account['ZI_Company_Revenue_Range__c'] == '$500,000 - $1 mil.'), 'Client_revenue'] = 'Under $1mil'
sales_account.loc[(sales_account['ZI_Company_Revenue_Range__c'] == '$1 mil. - $5 mil.') | (sales_account['ZI_Company_Revenue_Range__c'] == '$5 mil. - $10 mil.'), 'Client_revenue'] = '$1mil-$10mil'
sales_account.loc[(sales_account['ZI_Company_Revenue_Range__c'] == '$10 mil. - $25 mil.') | (sales_account['ZI_Company_Revenue_Range__c'] == '$25 mil. - $50 mil.')| (sales_account['ZI_Company_Revenue_Range__c'] == '$50 mil. - $100 mil.'), 'Client_revenue'] = '$10mil-$100mil'
sales_account.loc[(sales_account['ZI_Company_Revenue_Range__c'] == '$100 mil. - $250 mil.') | (sales_account['ZI_Company_Revenue_Range__c'] == '$250 mil. - $500 mil.'), 'Client_revenue'] = '$100mil-$500mil'
sales_account.loc[(sales_account['ZI_Company_Revenue_Range__c'] == '$500,000 - $1 mil.') | (sales_account['ZI_Company_Revenue_Range__c'] == '$1 bil. - $5 bil.') , 'Client_revenue'] = '$500mil-$5bil'
sales_account.loc[(sales_account['ZI_Company_Revenue_Range__c'] == 'Over $5 bil.'), 'Client_revenue'] = 'Over $5bil'

sales_consolidated_lasttransaction = sales_account.loc[sales_account['last'] == 'true']


## Importing Top selling products Information and building Item based features

In [14]:
top_sellers=top_sellers.fillna(0)
top_sellers['total_rev'] = top_sellers['2016'] + top_sellers['2017'] + top_sellers['2018'] + top_sellers['2019'] + top_sellers['2020']+ top_sellers['2021'] + top_sellers['2022']
top_sellers=top_sellers.fillna(0)

top_sellers.loc[top_sellers['total_rev'] >= 500000, 'item_type'] = 'Platinum'
top_sellers.loc[(top_sellers['total_rev'] >= 250000) & (top_sellers['total_rev'] < 500000), 'item_type'] = 'Gold'
top_sellers.loc[(top_sellers['total_rev'] >= 100000) & (top_sellers['total_rev'] < 250000), 'item_type'] = 'Silver'
top_sellers.loc[(top_sellers['total_rev'] >= 50000) & (top_sellers['total_rev'] < 100000), 'item_type'] = 'Bronze'
top_sellers.loc[(top_sellers['total_rev'] < 50000), 'item_type'] = 'Metal'
top_sellers.loc[top_sellers['total_rev'] >= 500000, 'Platinum'] = 1
top_sellers.loc[(top_sellers['total_rev'] >= 250000) & (top_sellers['total_rev'] < 500000), 'Gold'] = 1
top_sellers.loc[(top_sellers['total_rev'] >= 100000) & (top_sellers['total_rev'] < 250000), 'Silver'] = 1
top_sellers.loc[(top_sellers['total_rev'] >= 50000) & (top_sellers['total_rev'] < 100000), 'Bronze'] = 1
top_sellers.loc[(top_sellers['total_rev'] < 50000), 'Metal'] = 1
top_sellers=top_sellers.fillna(0)
top_items = top_sellers[['Row Labels','item_type','Platinum','Gold','Silver','Bronze','Metal']]
top_items = top_items.rename(columns={"Row Labels": "Item"})

#---------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx------------------------------------------#

sales = pd.merge(sales,
                 top_items,
                 on = ('Item'), 
                 how = 'left')

sales['year'] = pd.DatetimeIndex(sales['Posting Date']).year
sales = sales[sales['Platinum'].notnull()]
platinum_sales = sales.groupby('customer_time')['Platinum'].sum().reset_index()
sales_account['Platinum_products'] = platinum_sales['Platinum']
gold_sales = sales.groupby('customer_time')['Gold'].sum().reset_index()
sales_account['Gold_products'] = gold_sales['Gold']
Silver_sales = sales.groupby('customer_time')['Silver'].sum().reset_index()
sales_account['Silver_products'] = Silver_sales['Silver']
Bronze_sales = sales.groupby('customer_time')['Bronze'].sum().reset_index()
sales_account['Bronze_products'] = Bronze_sales['Bronze']
Metal_sales = sales.groupby('customer_time')['Metal'].sum().reset_index()
sales_account['Metal_products'] = Metal_sales['Metal']

sales_account['total_platinum_items'] = sales_account.groupby('Customer No')['Platinum_products'].cumsum()
sales_account['total_gold_items'] = sales_account.groupby('Customer No')['Gold_products'].cumsum()
sales_account['total_silver_items'] = sales_account.groupby('Customer No')['Silver_products'].cumsum()
sales_account['total_Bronze_items'] = sales_account.groupby('Customer No')['Bronze_products'].cumsum()
sales_account['total_Metal_items'] = sales_account.groupby('Customer No')['Metal_products'].cumsum()

#---------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx------------------------------------------#

sales_account['Platinum_Quality_Ratio'] = (sales_account['total_platinum_items'])/((sales_account['total_platinum_items'])
                                                                                   +sales_account['total_gold_items']
                                                                                   +sales_account['total_silver_items']
                                                                                   +sales_account['total_Bronze_items']
                                                                                   +sales_account['total_Metal_items'])*100

sales_account['Platinum_Gold_Quality_Ratio'] = (sales_account['total_platinum_items']+sales_account['total_gold_items'])/((sales_account['total_platinum_items'])
                                                                                   +sales_account['total_gold_items']
                                                                                   +sales_account['total_silver_items']
                                                                                   +sales_account['total_Bronze_items']
                                                                                   +sales_account['total_Metal_items'])*100

sales_account['Platinum_Quality_Ratio'] = sales_account['Platinum_Quality_Ratio']*100
sales_account['Platinum_Gold_Quality_Ratio'] = sales_account['Platinum_Gold_Quality_Ratio']*100
sales_account.loc[sales_account['total_platinum_items'] == 0, 'Platinum_bucket'] = '0'
sales_account.loc[(sales_account['total_platinum_items'] >= 1) & (sales_account['total_platinum_items'] < 5), 'Platinum_bucket'] = '1-5'
sales_account.loc[(sales_account['total_platinum_items'] >= 5), 'Platinum_bucket'] = '>5'

sales_consolidated_lasttransaction = sales_account.loc[(sales_account['last'] == 'true')]

#---------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx------------------------------------------#

sales_account.loc[sales_account['Platinum_Quality_Ratio'] == 0, 'Platinum_Quality_bucket'] = '0'
sales_account.loc[(sales_account['Platinum_Quality_Ratio'] >= 1) & (sales_account['Platinum_Quality_Ratio'] < 25), 'Platinum_Quality_bucket'] = '1-25'
sales_account.loc[(sales_account['Platinum_Quality_Ratio'] >= 25) & (sales_account['Platinum_Quality_Ratio'] <= 100), 'Platinum_Quality_bucket'] = '>25'
sales_consolidated_lasttransaction = sales_account.loc[(sales_account['last'] == 'true')]

#---------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx------------------------------------------#

first_sale = sales_account.loc[sales_account['count'] == 1][['Customer No','Posting Date']]
first_sale = first_sale.rename(columns={"Posting Date": "first_sale_date"})
sales_account = pd.merge(sales_account,
                 first_sale,
                 on=('Customer No'), 
                 how='left')
sales_account['tenure'] = sales_account['Posting Date'] - sales_account['first_sale_date']
sales_account['tenure'] = sales_account['tenure']/np.timedelta64(1,'D')
sales_consolidated_lasttransaction = sales_account.loc[(sales_account['last'] == 'true')]

### Feature Engineering

##### Average Quantity Ordered, Total Quantity, Gradient Quantity, Total Cost and Average Cost

In [15]:
#Master data set for features
features = sales_account
#1- Average Quantity of items ordered by the customer
features['AvgQuantity'] = (
    features.groupby('Customer No')['Quantity Ordered'].apply(lambda x: x.expanding().mean()))

#2- Total number of items purchased by the customer till date
features['TotalQuantity'] = (
    features.groupby('Customer No')['Quantity Ordered'].cumsum())

#3- Gradient of the number of products ordered by customer
features['GradientQuantity'] = (
    features.groupby('Customer No')['Quantity Ordered'].diff())

#5- Total revenue generated by the customer to date/ CLV
features['TotalCost'] = (
    features.groupby('Customer No')['Line Item Amount'].cumsum())

#6- Average revenue of the customer (Line amount/total orders) 
features['AvgCost'] = (
    features.groupby('Customer No')['Line Item Amount'].apply(lambda x: x.expanding().mean()))


#------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx---------------------------------------------#

##### Net Purchasing Interval Increasing or Decreasing Variable
sales_account.loc[sales_account['Customer No'] == sales_account['Customer No'].shift(1),'last_order_date'] = sales_account['Posting Date'].shift(1) 
sales_account['time_between_sales_trend'] = sales_account['Posting Date'] - sales_account['last_order_date']
sales_account['time_between_sales_trend'] = sales_account['time_between_sales_trend']/np.timedelta64(1,'D')
sales_account['time_between_sales_trend'] = (
    sales_account.groupby('Customer No')['time_between_sales_trend'].diff())
sales_account['time_between_sales_trend'] = (
    sales_account.groupby('Customer No')['time_between_sales_trend'].cumsum())


#------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx---------------------------------------------#


##### Net Sales Revenue Increasing or Decreasing
sales_account['Gradient_Revenue'] = (
    features.groupby('Customer No')['Line Item Amount'].diff())
sales_account['Gradient_Revenue'] = (
    sales_account.groupby('Customer No')['Gradient_Revenue'].cumsum())

#------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx---------------------------------------------#

# Customer Specific top Items
x=pd.DataFrame(sales.groupby(['Customer No'])['Item'].value_counts().rename('Count').reset_index()) 
x= x.drop_duplicates('Customer No')
features =pd.merge(features, x, on=('Customer No'), how='left')
features = features.rename(columns={'Item':'Customer_specific_pop.Item', 'Count':'Popular_item_count'})
y = pd.DataFrame(sales.groupby(['Customer No'])['Department Dim Description'].value_counts().rename('Count').reset_index()) 
y= y.drop_duplicates('Customer No')
features =pd.merge(features, y, on=('Customer No'), how='left')
features = features.rename(columns={'Department Dim Description':'top_department_dims', 'Count':'popular__department_dims_count'})
z = pd.DataFrame(sales.groupby(['Customer No'])['Product Group Description'].value_counts().rename('Count').reset_index()) 
z= z.drop_duplicates('Customer No')
features =pd.merge(features, z, on=('Customer No'), how='left')
features = features.rename(columns={'Product Group Description':'top_product_dims', 'Count':'popular_dims_count'})
sales_account = features

u = pd.DataFrame(sales.groupby(['Customer No','Department Dim Description'])['Item'].count().reset_index())
u['product_lines'] = pd.DataFrame(u[['Customer No','Department Dim Description']].groupby(['Customer No'])['Department Dim Description'].transform(lambda x : ' | '.join(x)))
u= u.drop_duplicates('Customer No')
u.drop(['Department Dim Description','Item'], axis = 1, inplace = True) 

#------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx---------------------------------------------#

'''
##### No. of Managers & Consolidated Silverpop score
contact1 = contact1.rename(columns={"AccountNumber": "Customer No"})
contact1 = contact1[['Customer No','No.Of Managers','Consolidated_Silverpop']]
sales_account = pd.merge(sales_account,
                 contact1,
                 on=('Customer No'), 
                 how='left')
'''
#------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx---------------------------------------------#


##### Percent DGOP contacts
contact2 = contact.groupby('Acct_Number__c').agg(NoofContacts_DGOP=('Job_Function_LM__c', 'value_counts')).reset_index()
contact3=contact2.groupby('Acct_Number__c').agg(TotalContacts=('NoofContacts_DGOP', 'sum')).reset_index()
contact_DGOP=contact2.loc[(contact2['Job_Function_LM__c']=="DG") | (contact2['Job_Function_LM__c']=="OP")]
contact_DGOP=contact_DGOP.groupby('Acct_Number__c').agg(Total_DGOP_Contacts=('NoofContacts_DGOP', 'sum')).reset_index()
contact_DGOP=contact_DGOP.merge(contact3,on=('Acct_Number__c'),how=('left'))
contact_DGOP['Perc_DGOP']=((contact_DGOP['Total_DGOP_Contacts'])/(contact_DGOP['TotalContacts'])*100)

sales_account = sales_account.merge(contact_DGOP,left_on=('Customer No'),right_on=('Acct_Number__c'),how=('left'))

#------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx---------------------------------------------#

##### Compliance score of family and family data join with sales_account
family['Family_Name__c'] = family['Family_Name__c'].astype(str)
family['Family_Segmentation__c'] = family['Family_Segmentation__c'].fillna('not_compliant')
sub ='Safely'
sub2 = 'Basically'
sub3 = 'not_compliant'
family.loc[family["Family_Segmentation__c"].str.contains(sub),'compliance_score'] = 1
family.loc[family["Family_Segmentation__c"].str.contains(sub2),'compliance_score'] = 2
family.loc[family["Family_Segmentation__c"].str.contains(sub3),'compliance_score'] = 3
del family['Tier_Note__c']
sales_account['family_code'] = sales_account['FamilyCode_FNT__c'].astype(str).str.split('.').str[0]
sales_account = sales_account.merge(family,left_on=('family_code'),right_on=('Family_Name__c'),how=('left'))

del sales_account['ZI_Industry__c']
industry = account[['Customer No','ZI_Industry__c']]
sales_account = sales_account.merge(industry,left_on=('Customer No'),right_on=('Customer No'),how=('left'))
sales_account.loc[sales_account['ZI_Industry__c'].isnull(),'ZI_Industry__c'] = sales_account['Family_Industry_Description_New__c']
sales_consolidated_lasttransaction = sales_account.loc[(sales_account['last'] == 'true')]

#------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx---------------------------------------------#

##### Item Diversity Index and Dept Diversity Index
sales_item = sales.groupby(['customer_time'])['Item'].nunique().reset_index()
sales_dept = sales.groupby('customer_time')['Department Dim Description'].nunique().reset_index()
sales_item = sales_item.rename(columns={'Item': "item_diversity_index"})
sales_dept = sales_dept.rename(columns={'Department Dim Description': "dept_diversity_index"})
sales_account = sales_account.merge(sales_item, on = ('customer_time'),how=('left'))
sales_account = sales_account.merge(sales_dept, on = ('customer_time'),how=('left'))
sales_account['cumulative_item_count'] = sales_account.groupby('Customer No')['item_diversity_index'].cumsum()
sales_account['cumulative_dept_count'] = sales_account.groupby('Customer No')['dept_diversity_index'].cumsum()
sales_account['item_diversity_index'] = sales_account['cumulative_item_count']/sales_account['count']
sales_account['dept_diversity_index'] = sales_account['cumulative_dept_count']/sales_account['count']
sales_account.drop(['cumulative_item_count', 'cumulative_dept_count'], axis=1, inplace=True)
sales_consolidated_lasttransaction = sales_account.loc[(sales_account['last'] == 'true')]
sales_consolidated_lasttransaction =pd.merge(sales_consolidated_lasttransaction, u, on=('Customer No'), how='left')

#------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx---------------------------------------------#

### Date Churned
sales_consolidated_lasttransaction['date_churned'] = sales_consolidated_lasttransaction['Posting Date'] +  pd.to_timedelta(sales_consolidated_lasttransaction['time_to_churn_standard'], unit='D')
EndDate = date.today() + timedelta(days=10)


## Output File for Marketing
output_data = sales_consolidated_lasttransaction[['Customer No','churn_standard','date_churned','Tier__c','cumulative_sum_revenue','ZI_Industry__c',
                                   'Perc_DGOP','product_lines','Customer_specific_pop.Item','top_product_dims',
                                    'top_department_dims']]


In [16]:
IIT_churn_master_dataset = sales_account
IIT_churn_customer_master_dataset = sales_consolidated_lasttransaction
IIT_churn_master_dataset = IIT_churn_master_dataset.drop(['Quantity Ordered','Line Item Amount','next_order_date','block',
                       'last','year','Id_x','CreatedDate_x','LastActivityDate','FamilyCode_FNT__c',
                       'Platinum_products','Gold_products','Silver_products','Bronze_products','Metal_products',
                       'first_sale_date','GradientQuantity','Acct_Number__c','family_code','Id_y','CreatedDate_y',
                       'Family_Name__c','last_order_date'],axis = 1)
IIT_churn_customer_master_dataset = IIT_churn_customer_master_dataset.drop(['customer_time','Quantity Ordered','Line Item Amount','next_order_date','block',
                       'last','year','Id_x','CreatedDate_x','LastActivityDate','FamilyCode_FNT__c','count_group',
                       'Platinum_products','Gold_products','Silver_products','Bronze_products','Metal_products',
                       'first_sale_date','GradientQuantity','Acct_Number__c','family_code','Id_y','CreatedDate_y',
                       'Family_Name__c','last_order_date'],axis = 1)

# Replacing Missing values
IIT_churn_master_dataset['Client_revenue'] = sales_account['Client_revenue']
IIT_churn_master_dataset['Client_revenue'].fillna('other', inplace = True)
IIT_churn_master_dataset['Family_Industry_Description_New__c'].fillna('Unassigned', inplace = True)
IIT_churn_master_dataset['compliance_score'].fillna(3, inplace = True)
IIT_churn_master_dataset["compliance_score"].replace({1: "High", 2: "Medium", 3:"Low"}, inplace=True)
IIT_churn_master_dataset['Territory__c'].fillna('UNKNOWN', inplace = True)
IIT_churn_master_dataset['plat_gold_per_transaction'] = (IIT_churn_master_dataset['total_platinum_items'] + IIT_churn_master_dataset['total_gold_items'])/IIT_churn_master_dataset['count']
IIT_churn_master_dataset['Perc_DGOP'].fillna(33, inplace = True)

# Making buckets of gradient rev
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['Gradient_Revenue'] <= 0),'gradient_rev_direction'] = 'negative'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['Gradient_Revenue'] > 0) & (IIT_churn_master_dataset['Gradient_Revenue'] < 500),'gradient_rev_direction'] = 'positive'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['Gradient_Revenue'] >= 500),'gradient_rev_direction'] = 'very_positive'

# Making buckets of item diversity index
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['item_diversity_index'] > 0) & (IIT_churn_master_dataset['item_diversity_index'] <= 2),'item_diversity_buckets'] = '1-2'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['item_diversity_index'] > 2) & (IIT_churn_master_dataset['item_diversity_index'] <= 4),'item_diversity_buckets'] = '2-4'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['item_diversity_index'] > 4) & (IIT_churn_master_dataset['item_diversity_index'] <= 6),'item_diversity_buckets'] = '4-6'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['item_diversity_index'] > 6) & (IIT_churn_master_dataset['item_diversity_index'] <= 8),'item_diversity_buckets'] = '6-8'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['item_diversity_index'] > 8),'item_diversity_buckets'] = '>8'

# Making buckets of time between Sales
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['time_between_sales_trend'] < -50) ,'time_between_sales_trend_buckets'] = '<-30'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['time_between_sales_trend'] > -50) & (IIT_churn_master_dataset['time_between_sales_trend'] <= 50),'time_between_sales_trend_buckets'] = '-30_to_30'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['time_between_sales_trend'] > 50) & (IIT_churn_master_dataset['time_between_sales_trend'] <= 100),'time_between_sales_trend_buckets'] = '30-100'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['time_between_sales_trend'] > 100),'time_between_sales_trend_buckets'] = '>100'

# Making buckets of DGOP
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['Perc_DGOP'] < 34 ),'Perc_DGOP_buckets'] = '<34'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['Perc_DGOP'] > 34) & (IIT_churn_master_dataset['Perc_DGOP'] <= 66),'Perc_DGOP_buckets'] = '34-66'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['Perc_DGOP'] > 66),'Perc_DGOP_buckets'] = '>66'

# Making buckets of dept_diversity_index
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['dept_diversity_index'] == 1),'dept_diversity_index_buckets'] = '1'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['dept_diversity_index'] > 1) & (IIT_churn_master_dataset['dept_diversity_index'] <= 1.5),'dept_diversity_index_buckets'] = '1-1.5'
IIT_churn_master_dataset.loc[(IIT_churn_master_dataset['dept_diversity_index'] > 1.5) ,'dept_diversity_index_buckets'] = '>1.5'


### Final Dataset preparation

In [36]:
IIT_churn_master_dataset_2 = IIT_churn_master_dataset.drop(['customer_time','Posting Date','time_to_order',
                       'expected_purchase_window','time_to_churn_standard','time_to_churn_modified','churn_modified','churn_score','churn alarm',
                       'HQ_State__c','ZI_Technologies__c','total_platinum_items','total_gold_items',
                       'total_silver_items','total_Bronze_items','total_Metal_items','Platinum_bucket','Customer_specific_pop.Item','BDM_and_Inside_Coverage__c',
                       'ZI_Company_Revenue_Range__c','ZI_Industry__c','Family_Segmentation__c','count_group',
                        'item_diversity_buckets','time_between_sales_trend_buckets','Perc_DGOP_buckets',
                        'dept_diversity_index_buckets','ZI_Company_Ranking__c','ZI_Company_Employee_Range__c',
                         'Total_DGOP_Contacts'],axis = 1)

IIT_churn_master_dataset_2 = IIT_churn_master_dataset_2.rename(columns={"Customer No": "customer_no"})##Imputing Missing Values

IIT_churn_master_dataset_2['ZI_Company_Type__c'].fillna('Unknown', inplace = True)
IIT_churn_master_dataset_2['ZI_Number_Of_Locations__c'].fillna(IIT_churn_master_dataset_2['ZI_Number_Of_Locations__c'].mode()[0], inplace = True)
IIT_churn_master_dataset_2['Gradient_Revenue'].fillna(0, inplace = True)
IIT_churn_master_dataset_2['time_between_sales_trend'].fillna(0, inplace = True)
IIT_churn_master_dataset_2['gradient_rev_direction'].fillna('Unknown', inplace = True)
IIT_churn_master_dataset_2['TotalContacts'].fillna(IIT_churn_master_dataset_2['TotalContacts'].mode()[0], inplace = True)
IIT_churn_master_dataset_2['Popular_item_count'].fillna(0, inplace = True)
IIT_churn_master_dataset_2['item_diversity_index'].fillna(0, inplace = True)
IIT_churn_master_dataset_2['Coverage_Priority__c'].fillna('Uncovered', inplace = True)

In [37]:
IIT_churn_master_dataset_2['Coverage_Priority__c'] = IIT_churn_master_dataset_2['Coverage_Priority__c'].astype('category')
IIT_churn_master_dataset_2['ZI_Company_Type__c'] = IIT_churn_master_dataset_2['ZI_Company_Type__c'].astype('category')
IIT_churn_master_dataset_2['Platinum_Quality_bucket'] = IIT_churn_master_dataset_2['Platinum_Quality_bucket'].astype('category')
IIT_churn_master_dataset_2['Family_Industry_Description_New__c'] = IIT_churn_master_dataset_2['Family_Industry_Description_New__c'].astype('category')
IIT_churn_master_dataset_2['Territory__c'] = IIT_churn_master_dataset_2['Territory__c'].astype('category')
IIT_churn_master_dataset_2['compliance_score'] = IIT_churn_master_dataset_2['compliance_score'].astype('category')

train = IIT_churn_master_dataset_2[['Coverage_Priority__c', 'ZI_Company_Type__c','count',
       'ZI_Number_Of_Locations__c', 'cumulative_sum_revenue', 
       'Platinum_Gold_Quality_Ratio','tenure', 'AvgCost', 'AvgQuantity','Popular_item_count', 'No.Of Managers',
       'Consolidated_Silverpop', 'TotalContacts', 'Perc_DGOP',
       'Family_Industry_Description_New__c', 'Territory__c',
       'compliance_score', 'Gradient_Revenue', 'time_between_sales_trend',
       'item_diversity_index', 'Client_revenue',
       'plat_gold_per_transaction','customer_no','churn_standard','Tier__c']]

train['last'] = sales_account['last']
IIT_churn_customer_master_dataset_2 = train.loc[(train['last'] == 'true')]
IIT_churn_customer_master_dataset_2=IIT_churn_customer_master_dataset_2.drop(['last'],axis = 1)
train=train.drop(['last'],axis = 1)

<ipython-input-37-ddb77808885b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last'] = sales_account['last']


# Model Building

In [38]:
Transaction_final_data = train
customer = IIT_churn_customer_master_dataset_2
customer = customer.reset_index(drop=True)

#Transaction_final_data=Transaction_final_data.drop(['Unnamed: 0','Unnamed: 0.1'],axis = 1)
#customer=customer.drop(['Unnamed: 0.1','Unnamed: 0'],axis = 1)
#output_data.drop('Unnamed: 0',axis = 1, inplace = True)
output_data = output_data.rename(columns={"Customer No": "customer_no"})

## Pre-Processing Transaction Data

In [62]:
## Categorical Variables
df3 = Transaction_final_data[['Coverage_Priority__c', 'ZI_Company_Type__c', 'Family_Industry_Description_New__c',
       'Territory__c', 'Client_revenue','compliance_score']].copy()

one_hot_encoded_data = pd.get_dummies(df3)

## Numerical Variables
df2 = Transaction_final_data[['count', 'ZI_Number_Of_Locations__c', 'cumulative_sum_revenue',
       'Platinum_Gold_Quality_Ratio', 'tenure','AvgCost','AvgQuantity','Popular_item_count',
        'Perc_DGOP','Gradient_Revenue','time_between_sales_trend',
        'item_diversity_index','plat_gold_per_transaction','customer_no']].copy()

tiers_exclude = ['GOV','','DISTST','GOVCON']

ss = StandardScaler()
x_scaled= pd.DataFrame(ss.fit_transform(df2.drop('customer_no', axis=1)))
x_scaled.columns = df2.drop('customer_no', axis=1).columns
x_transaction = pd.concat([x_scaled, one_hot_encoded_data, Transaction_final_data['customer_no'],Transaction_final_data['churn_standard'],Transaction_final_data['Tier__c']], axis=1)
x_transaction = x_transaction[~x_transaction.Tier__c.isin(tiers_exclude)]
x_transaction = x_transaction.drop('Tier__c', axis=1)
y_transaction = x_transaction['churn_standard']

## Pre-Processing Customer Data

In [64]:
## Categorical Variables
df1 = customer[['Coverage_Priority__c', 'ZI_Company_Type__c', 'Family_Industry_Description_New__c',
       'Territory__c', 'Client_revenue','compliance_score']].copy()

one_hot_encoded_data_c = pd.get_dummies(df1)

## Numerical Variables
df4 = customer[['count', 'ZI_Number_Of_Locations__c', 'cumulative_sum_revenue',
       'Platinum_Gold_Quality_Ratio', 'tenure','AvgCost','AvgQuantity','Popular_item_count',
        'Perc_DGOP','Gradient_Revenue','time_between_sales_trend',
        'item_diversity_index','plat_gold_per_transaction','customer_no']].copy()

x_scaled_c= pd.DataFrame(ss.fit_transform(df4.drop('customer_no', axis=1)))
x_scaled_c.columns = df4.drop('customer_no', axis=1).columns
x_transaction_c = pd.concat([x_scaled_c, one_hot_encoded_data_c, customer['Tier__c'],customer['customer_no'],customer['churn_standard']], axis=1)
x_transaction_c = x_transaction_c[~x_transaction_c.Tier__c.isin(tiers_exclude)]
x_transaction_c = x_transaction_c.drop('Tier__c', axis=1)
y_transaction_c = x_transaction_c['churn_standard']

In [73]:
# Renaming Columns
x_transaction_c = x_transaction_c.rename(columns={"count":"Count_Of_Transactions","ZI_Number_Of_Locations__c":"Number_of_Locations","cumulative_sum_revenue":"Cumulative_Sum_Of_Revenue","tenure":"Customer_Tenure","AvgCost":"Average_Cost_Of_Transactions","AvgQuantity":"Average_Quantity_Of_Items","Popular_item_count":"Count_Of_Popular_Items","Consolidated_Silverpop":"Consolidated_Customer_Silver_Pop_Score","Perc_DGOP":"Percentage_Of_DG_OP_Contacts","Gradient_Revenue":"Change_In_Revenue","time_between_sales_trend":"Duration_Between_Transactions","item_diversity_index":"Index_for_Item_Diversity","plat_gold_per_transaction":"Platinum_and_Gold_Items_per_Transactions","Coverage_Priority__c_Amazon":"Coverage_Priority_for_Amazon","Coverage_Priority__c_BDM":"Coverage_Priority_for_BDM","Coverage_Priority__c_Distributor":"Coverage_Priority_for_Distributor",'Coverage_Priority__c_Government':'Coverage_Priority_for_Government','Coverage_Priority__c_Government Contractor':'Coverage_Priority_for_Government_Contractor','Coverage_Priority__c_Inside Sales':'Coverage_Priority_for_Inside_Sales','Coverage_Priority__c_Internal Customer':'Coverage_Priority_for_Internal_Customer','Coverage_Priority__c_Spacemaster':'Coverage_Priority_for_Spacemaster','Coverage_Priority__c_Strategic Distributor':'Coverage_Priority_for_Strategic_Distributor','Coverage_Priority__c_Uncovered':'Coverage_Priority_for_Uncovered','ZI_Company_Type__c_EDUCATION':'Company_Type_as_EDUCATION','ZI_Company_Type__c_GOVERNMENT':'Company_Type_as_GOVERNMENT','ZI_Company_Type__c_NPO':'Company_Type_as_NPO','ZI_Company_Type__c_OTHER':'Company_Type_as_OTHER','ZI_Company_Type__c_PRIVATE':'Company_Type_as_PRIVATE','ZI_Company_Type__c_PUBLIC':'Company_Type_as_PUBLIC','ZI_Company_Type__c_Private':'Company_Type_as_Private','ZI_Company_Type__c_Unknown':'Company_Type_as_UNKNOWN','Family_Industry_Description_New__c_Agriculture & Food':'Family_Industry_as_Agriculture_&_Food','Family_Industry_Description_New__c_Automotive':'Family_Industry_as_Automotive','Family_Industry_Description_New__c_Aviation & Aerospace':'Family_Industry_as_Aviation_&_Aerospace','Family_Industry_Description_New__c_Chemical':'Family_Industry_as_Chemical','Family_Industry_Description_New__c_Education':'Family_Industry_as_Education','Family_Industry_Description_New__c_Gov & Military':'Family_Industry_as_Govt_&_Military','Family_Industry_Description_New__c_Mining Oil & Gas':'Family_Industry_as_MiningOil_&_Gas','Family_Industry_Description_New__c_Other':'Family_Industry_as_Other','Family_Industry_Description_New__c_Other Manufacturing':'Family_Industry_as_Other_Manufacturing','Family_Industry_Description_New__c_Pharma & Healthcare':'Family_Industry_as_Pharma_&_Healthcare','Family_Industry_Description_New__c_Retail Trade':'Family_Industry_as_Retail_Trade','Family_Industry_Description_New__c_Services':'Family_Industry_as_Services','Family_Industry_Description_New__c_Technology':'Family_Industry_as_Technology','Family_Industry_Description_New__c_Transport & Warehouse':'Family_Industry_as_Transport_&_Warehouse','Family_Industry_Description_New__c_Transport Equipment':'Family_Industry_as_Transport_Equipment','Family_Industry_Description_New__c_Unassigned':'Family_Industry_as_Unassigned','Family_Industry_Description_New__c_WasteManagement':'Family_Industry_as_Waste_Management','Family_Industry_Description_New__c_Wholesale Trade':'Family_Industry_as_Wholesale_Trade','Territory__c_CANADA':'Territory_CANADA','Territory__c_GREAT LAKES':'Territory_GREAT_LAKES','Territory__c_MIDWEST':'Territory_MIDWEST','Territory__c_Mid-Atlantic':'Territory_MID_ATLANTIC','Territory__c_NORTHEAST':'Territory_NORTHEAST', 'Territory__c_SOUTHEAST':'Territory_SOUTHEAST','Territory__c_UNKNOWN':'Territory_UNKNOWN','Territory__c_WEST':'Territory_WEST','Client_revenue_$100mil-$500mil':'Client_Revenue_Range_$100mil-$500mil','Client_revenue_$10mil-$100mil':'Client_Revenue_Range_$10mil-$100mil','Client_revenue_$1mil-$10mil':'Client_Revenue_Range_$1mil-$10mil','Client_revenue_$500mil-$5bil':'Client_Revenue_Range_$500mil-$5bil','Client_revenue_Over $5bil':'Client_Revenue_Range_Over_$5bil','Client_revenue_Under$1mil':'Client_Revenue_Range_Under$1mil','Client_revenue_other':'Client_Revenue_Range_Other','compliance_score_High':'Compliance_Score_as_High', 'compliance_score_Low':'Compliance_Score_as_Low','compliance_score_Medium':'Compliance_Score_as_Medium'})


## Train-Test Split

In [74]:
X_train, X_test, y_train, y_test = train_test_split(x_transaction_c,y_transaction_c, test_size=0.1,random_state=10, stratify=y_transaction_c)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.222, random_state=11, stratify=y_train)
# Creating training set from transaction wise data
X_train_transaction = x_transaction[x_transaction['customer_no'].isin(X_train['customer_no'])]
y_train_transaction = X_train_transaction['churn_standard']

X_train = X_train.dropna()
X_val = X_val.dropna()
X_test = X_test.dropna()
y_train = X_train['churn_standard']
y_val = X_val['churn_standard']
y_test = X_test['churn_standard']

X_train_transaction.drop(['churn_standard'], axis=1, inplace=True)
X_train.drop(['churn_standard'], axis=1, inplace=True)
X_val.drop(['churn_standard'], axis=1, inplace=True)
X_test.drop(['churn_standard'], axis=1, inplace=True)

<ipython-input-74-2e6c2f3e9b7a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_transaction.drop(['churn_standard'], axis=1, inplace=True)


## XGBoost


### Model Fitting

In [75]:
# Implementing XGBoosting and Gradient boosting with hyperparameter tuning
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=6 , use_label_encoder =False,
    eval_metric='logloss',
    learning_rate = 0.1,
    gamma = 0.9
)

X_train_c = X_train.loc[:, X_train.columns != 'customer_no']
X_val_c = X_val.loc[:, X_val.columns != 'customer_no']
X_test_c = X_test.loc[:, X_test.columns != 'customer_no']
clf = estimator.fit(X_train_c, y_train)

### Validation Data
y_pred = clf.predict(X_val_c)

### Test Data
y_pred = clf.predict(X_test_c)

C:\Users\92306\anaconda3\lib\site-packages\xgboost\data.py:200: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\92306\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [76]:
## XGBOOST Data Preparation for final dataset

yhat = clf.predict_proba(X_train_c)
probs_train = yhat[:, 1]
X_train['churn_probability_XGBoost'] = probs_train
#X_train_xg = X_train[['customer_no','churn_probability_XGBoost']]

yhat = clf.predict_proba(X_val_c)
probs_val = yhat[:, 1]
X_val['churn_probability_XGBoost'] = probs_val
#X_val_xg = X_val[['customer_no','churn_probability_XGBoost']]

yhat = clf.predict_proba(X_test_c)
probs_test = yhat[:, 1]
X_test['churn_probability_XGBoost'] = probs_test
#X_test_xg = X_test[['customer_no','churn_probability_XGBoost']]

X_train_xg = X_train[['customer_no','churn_probability_XGBoost']]
X_val_xg = X_val[['customer_no','churn_probability_XGBoost']]
X_test_xg = X_test[['customer_no','churn_probability_XGBoost']]
xg_results = pd.concat([X_train_xg,X_val_xg,X_test_xg])

C:\Users\92306\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\92306\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


## Logistic Regression

### Model Fitting

In [77]:
X_train_na = X_train.dropna()
X_val_na = X_val.dropna()
X_test_na = X_test.dropna()

X_train_sub = X_train_na[['Count_Of_Transactions','Customer_Tenure','Cumulative_Sum_Of_Revenue','Platinum_Gold_Quality_Ratio','Count_Of_Popular_Items',
                      'Change_In_Revenue','Duration_Between_Transactions','Index_for_Item_Diversity'
                       ,'Coverage_Priority_for_Uncovered','customer_no']]

X_val_sub = X_val_na[['Count_Of_Transactions','Customer_Tenure','Cumulative_Sum_Of_Revenue','Platinum_Gold_Quality_Ratio','Count_Of_Popular_Items',
                      'Change_In_Revenue','Duration_Between_Transactions','Index_for_Item_Diversity'
                       ,'Coverage_Priority_for_Uncovered','customer_no']]

X_test_sub = X_test_na[['Count_Of_Transactions','Customer_Tenure','Cumulative_Sum_Of_Revenue','Platinum_Gold_Quality_Ratio','Count_Of_Popular_Items',
                      'Change_In_Revenue','Duration_Between_Transactions','Index_for_Item_Diversity'
                       ,'Coverage_Priority_for_Uncovered','customer_no']]

X_train_c = X_train_sub.loc[:, X_train_sub.columns != 'customer_no']
X_val_c = X_val_sub.loc[:, X_val_sub.columns != 'customer_no']
X_test_c = X_test_sub.loc[:, X_test_sub.columns != 'customer_no']

logit = LogisticRegression( penalty = 'l2',max_iter = 500)
logit = logit.fit(X_train_c, y_train)

### Validation Data
y_pred = logit.predict(X_val_c)

### Test Data
y_pred = logit.predict(X_test_c)


### Feature Importance Logistic

In [79]:
model_1_features = X_train_c.columns.values.tolist()
logit_coef = logit.coef_[0]

importances = pd.DataFrame(data={
    'Attribute': X_train_c.columns,
    'Importance': abs(logit.coef_[0])
})
importances = importances.sort_values(by='Importance', ascending=False)

cov_contribution = abs(X_train_c.values*logit_coef)
cov_contribution = pd.DataFrame(cov_contribution, columns = X_train_c.columns)
cov_contribution.drop(['Change_In_Revenue','Platinum_Gold_Quality_Ratio'], axis=1, inplace=True)
res =cov_contribution.div(cov_contribution.sum(axis=1), axis=0)
res = res.mul(100)
res.drop('churn_standard' , axis = 1, inplace = True)

## Model Results for Output File

In [109]:
X_train_sub = X_train_sub.reset_index(drop=True)
res['customer_no'] = X_train_sub['customer_no']
yhat = logit.predict_proba(X_train_c)
probs_train = yhat[:, 1]
res['churn_probability_logit'] = probs_train

###  val data predictions/probability

cov_contribution = abs(X_val_c.values*logit_coef)
cov_contribution = pd.DataFrame(cov_contribution, columns = X_val_c.columns)
cov_contribution.drop(['Change_In_Revenue','Platinum_Gold_Quality_Ratio'], axis=1, inplace=True)
res_val =cov_contribution.div(cov_contribution.sum(axis=1), axis=0)
res_val = res_val.mul(100)

X_val_sub = X_val_sub.reset_index(drop=True)
res_val['customer_no'] = X_val_sub['customer_no']
yhat = logit.predict_proba(X_val_c)
probs_val = yhat[:, 1]
res_val['churn_probability_logit'] = probs_val


###  test data predictions/probability

cov_contribution = abs(X_test_c.values*logit_coef)
cov_contribution = pd.DataFrame(cov_contribution, columns = X_test_c.columns)
cov_contribution.drop(['Change_In_Revenue','Platinum_Gold_Quality_Ratio'], axis=1, inplace=True)
res_test =cov_contribution.div(cov_contribution.sum(axis=1), axis=0)
res_test = res_test.mul(100)

X_test_sub = X_val_sub.reset_index(drop=True)
res_test['customer_no'] = X_test_sub['customer_no']
yhat = logit.predict_proba(X_test_c)
probs_test = yhat[:, 1]
res_test['churn_probability_logit'] = probs_test

model_results = pd.concat([res,res_val,res_test])
model_results =pd.merge(model_results, xg_results, on=('customer_no'), how='left')
model_results =pd.merge(model_results, output_data, on=('customer_no'), how='left')

## Final Table to be used in power BI

In [111]:
model_results

,Count_Of_Transactions,Customer_Tenure,Cumulative_Sum_Of_Revenue,Count_Of_Popular_Items,Duration_Between_Transactions,Index_for_Item_Diversity,Coverage_Priority_for_Uncovered,customer_no,churn_probability_logit,churn_probability_XGBoost,churn_standard,date_churned,Tier__c,cumulative_sum_revenue,ZI_Industry__c,Perc_DGOP,product_lines,Customer_specific_pop.Item,top_product_dims,top_department_dims
0,1.416406,88.984552,0.368756,0.012161,2.938137,2.164204,4.115782,0400128,0.073383,0.027979,0.0,2022-07-24 20:34:17.142844800,TIER8,3595.24,Manufacturing,100.000000,Facility Management | Labels | Software,DGISEXPSS,Masterform New,Software
1,9.952920,40.837464,9.860276,0.066614,1.069971,15.668578,22.544177,0348205,0.465341,0.697551,1.0,2020-12-04 07:59:59.999971200,TIER8,33901.20,Transportation,NaN,Books | Placards,IMO0027,Placard Accessories,Placards
2,3.372399,84.798515,1.810304,0.682040,2.567718,6.769024,0.000000,3055848,0.713519,0.730963,0.0,2022-06-14 00:00:00.000000000,TIER3-Pioneer,1181.98,Retail;Software,NaN,Books | Labels | Personal Compliance,SLQMM,DOT Labels,Labels
3,4.876262,84.675970,0.789595,0.340433,0.454108,3.634402,5.229230,7001365,0.893625,0.996270,1.0,2018-09-22 00:00:00.000000000,TIER8,64.10,Hospitals & Physicians Clinics;Organizations,NaN,Books | Placards,DOT0146,Code of Federal Regulations,Books
4,4.926180,87.870709,0.758380,0.343918,0.458757,0.359296,5.282761,1552759,0.889634,0.997429,1.0,2019-07-27 00:00:00.000000000,TIER8,491.91,Manufacturing,100.000000,Labels | Personal Compliance,WM8,Personalized Waste Lables,Labels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33181,0.548982,90.712484,0.352413,0.046755,1.971397,2.651950,3.716020,0266827,0.043799,0.463788,1.0,2019-09-22 00:00:00.000000000,TIER4-Explorer,912.01,Manufacturing,NaN,Books | Labels,F07LB,Forms,Books
33182,1.758810,89.810361,0.433152,0.050125,3.692752,0.270953,3.983848,0405673,0.052515,0.602499,0.0,2022-06-09 00:00:00.000000000,TIER7,3361.33,Manufacturing,NaN,Labels | Personal Compliance,SLB435P-00033600,Personalized Regulatory Markings,Personal Compliance
33183,4.755753,84.295621,0.756173,0.411257,0.442885,4.238314,5.099997,3064355,0.878710,0.813025,1.0,2022-02-20 00:00:00.000000000,TIER8,1137.91,Education,NaN,Books | Labels | Placards,IATA0205,DOT Labels,Labels
33184,2.307302,92.342547,0.400234,0.012633,0.371264,0.290772,4.275249,0348447,0.064026,0.258024,0.0,2022-08-23 02:10:54.545462400,TIER8,4483.08,Transportation,50.000000,Books | Imprinted Placards | Placards,LQMR,Orange Panels & Markings,Placards
